In [1]:
#Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from scipy import stats

In [2]:
# Store filepath in a variable
OPTN_data = Path("Resources/cleaned_OPTN_data.csv")

# read the file - AP
kidney_data = pd.read_csv(OPTN_data)

kidney_data.head()

,State,Total,< 30 days,30 to < 90 days,90 days to < 6 months,6 months to < 1 year,1 year to < 2 years,2 years to < 3 years,3 years to < 5 years,> 5 years,Payment type,Blood type,Age,Ethnicity,Sex
0,Alabama,455,9,22,61,101,146,73,54,40,NaN,NaN,NaN,NaN,Female
1,Alabama,713,16,38,87,150,237,135,95,43,NaN,NaN,NaN,NaN,Male
2,Alabama,787,5,7,72,169,267,146,97,48,Public insurance - Medicare FFS (Fee for Service),NaN,NaN,NaN,NaN
3,Alabama,688,9,32,88,154,215,122,93,57,NaN,NaN,NaN,"Black, Non-Hispanic",NaN
4,Alabama,631,14,30,80,136,209,124,81,46,NaN,O,NaN,NaN,NaN


In [3]:
# ignoring commas

kidney_data["Total"] = kidney_data["Total"].str.replace(",", "")
kidney_data["< 30 days"] = kidney_data["< 30 days"].str.replace(",", "")
kidney_data["30 to < 90 days"] = kidney_data["30 to < 90 days"].str.replace(",", "")
kidney_data["90 days to < 6 months"] = kidney_data["90 days to < 6 months"].str.replace(",", "")
kidney_data["6 months to < 1 year"] = kidney_data["6 months to < 1 year"].str.replace(",", "")
kidney_data["1 year to < 2 years"] = kidney_data["1 year to < 2 years"].str.replace(",", "")
kidney_data["2 years to < 3 years"] = kidney_data["2 years to < 3 years"].str.replace(",", "")
kidney_data["3 years to < 5 years"] = kidney_data["3 years to < 5 years"].str.replace(",", "")
kidney_data["> 5 years"] = kidney_data["> 5 years"].str.replace(",", "")

In [4]:
# need to type total and wait time columns as integers
kidney_data["Total"] = kidney_data["Total"].astype(int)
kidney_data["< 30 days"] = kidney_data["< 30 days"].astype(int)
kidney_data["30 to < 90 days"] = kidney_data["30 to < 90 days"].astype(int)
kidney_data["90 days to < 6 months"] = kidney_data["90 days to < 6 months"].astype(int)
kidney_data["6 months to < 1 year"] = kidney_data["6 months to < 1 year"].astype(int)
kidney_data["1 year to < 2 years"] = kidney_data["1 year to < 2 years"].astype(int)
kidney_data["2 years to < 3 years"] = kidney_data["2 years to < 3 years"].astype(int)
kidney_data["3 years to < 5 years"] = kidney_data["3 years to < 5 years"].astype(int)
kidney_data["> 5 years"] = kidney_data["> 5 years"].astype(int)

In [5]:
# testing
# variable = kidney_data.loc[(kidney_data["Sex"] == "Female") & (kidney_data["State"] == "Oklahoma"), "Total"].values[0]
# variable

### Oklahoma

In [6]:
# Total candidates for Oklahoma

ok_totals1 = kidney_data.loc[kidney_data["State"] == "Oklahoma"]
ok_male_totals = ok_totals1.loc[ok_totals1["Sex"] == "Male"]["Total"].values[0]
ok_female_totals = ok_totals1.loc[ok_totals1["Sex"] == "Female"]["Total"].values[0]
ok_totals = ok_male_totals + ok_female_totals
ok_totals

597

In [7]:
# Need to get wait time values for each blood type 

# Total for each blood type

ok_blood_type_O = ok_totals1.loc[ok_totals1["Blood type"] == "O"]["Total"].values[0]
ok_blood_type_A = ok_totals1.loc[ok_totals1["Blood type"] == "A"]["Total"].values[0]
ok_blood_type_B = ok_totals1.loc[ok_totals1["Blood type"] == "B"]["Total"].values[0]
ok_blood_type_AB = ok_totals1.loc[ok_totals1["Blood type"] == "AB"]["Total"].values[0]
print(ok_blood_type_O)
print(ok_blood_type_A)
print(ok_blood_type_B)
print(ok_blood_type_AB)

341
188
65
3


In [8]:
# Testing

#type_O_wait1 = ok_totals1.loc[ok_totals1["Blood type"] == "O"]["< 30 days"].values[0]
#type_O_wait1


In [9]:
# Creating blood type vs wait time dataframe

ok_blood_type_wait = pd.DataFrame(ok_totals1[["Blood type","< 30 days", "30 to < 90 days", "90 days to < 6 months", 
                                        "6 months to < 1 year", "1 year to < 2 years","2 years to < 3 years", 
                                        "3 years to < 5 years", "> 5 years"]]).dropna()
ok_blood_type_wait

,Blood type,< 30 days,30 to < 90 days,90 days to < 6 months,6 months to < 1 year,1 year to < 2 years,2 years to < 3 years,3 years to < 5 years,> 5 years
1067,O,16,24,39,64,82,70,48,15
1071,A,14,15,21,46,47,25,15,7
1078,B,3,9,7,13,13,8,9,5
1088,AB,0,0,1,0,1,1,0,0


In [10]:
#Drop nan rows
ok_blood_type_wait = ok_blood_type_wait.dropna()

#Reshape the data with melt for analysis.
melted_dataok = ok_blood_type_wait.melt(id_vars=["Blood type"], var_name="WaitTime", value_name="Days")

#Try dropping nan values again? This didn't work because there are too many zero, null values
melted_dataok = melted_dataok.dropna()

#Group data by blood type and wait time categories
grouped_dataok = melted_dataok.groupby(["Blood type", "WaitTime"])

#Statistical analysis for each group
stats_by_bloodtype = grouped_dataok["Days"].agg(["mean", "median", "var", "std", "sem"])

#Display
print(stats_by_bloodtype)

                                  mean  median  var  std  sem
Blood type WaitTime                                          
A          1 year to < 2 years    47.0    47.0  NaN  NaN  NaN
           2 years to < 3 years   25.0    25.0  NaN  NaN  NaN
           3 years to < 5 years   15.0    15.0  NaN  NaN  NaN
           30 to < 90 days        15.0    15.0  NaN  NaN  NaN
           6 months to < 1 year   46.0    46.0  NaN  NaN  NaN
           90 days to < 6 months  21.0    21.0  NaN  NaN  NaN
           < 30 days              14.0    14.0  NaN  NaN  NaN
           > 5 years               7.0     7.0  NaN  NaN  NaN
AB         1 year to < 2 years     1.0     1.0  NaN  NaN  NaN
           2 years to < 3 years    1.0     1.0  NaN  NaN  NaN
           3 years to < 5 years    0.0     0.0  NaN  NaN  NaN
           30 to < 90 days         0.0     0.0  NaN  NaN  NaN
           6 months to < 1 year    0.0     0.0  NaN  NaN  NaN
           90 days to < 6 months   1.0     1.0  NaN  NaN  NaN
        

In [13]:
#Experimentation with turning wait times into numeric values aka days
timeframe_mapping = {
    "< 30 days": 15,
    "30 to < 90 days": 60,
    "90 days to < 6 months": 120,
    "6 months to < 1 year": 180,
    "1 year to < 2 years": 365,
    "2 years to < 3 years": 730,
    "3 years to < 5 years": 1460,
    "> 5 years": 1825}

# Iterate over each wait time column and create corresponding numeric columns
for column in ["< 30 days", "30 to < 90 days", "90 days to < 6 months",
               "6 months to < 1 year", "1 year to < 2 years", "2 years to < 3 years",
               "3 years to < 5 years", "> 5 years"]:
    ok_blood_type_wait[f"Numeric_{column}"] = ok_blood_type_wait[column].map(timeframe_mapping)
    
print("\nAfter Mapping:")
print(ok_blood_type_wait)


After Mapping:
     Blood type  < 30 days  30 to < 90 days  90 days to < 6 months  \
1067          O         16               24                     39   
1071          A         14               15                     21   
1078          B          3                9                      7   
1088         AB          0                0                      1   

      6 months to < 1 year  1 year to < 2 years  2 years to < 3 years  \
1067                    64                   82                    70   
1071                    46                   47                    25   
1078                    13                   13                     8   
1088                     0                    1                     1   

      3 years to < 5 years  > 5 years  Numeric_< 30 days  \
1067                    48         15                NaN   
1071                    15          7                NaN   
1078                     9          5                NaN   
1088                     0 

In [ ]:
# Plot above on bar chart

In [ ]:
# Creating sex vs wait time dataframe

ok_sex_wait = pd.DataFrame(ok_totals1[["Sex","< 30 days", "30 to < 90 days", "90 days to < 6 months", 
                                        "6 months to < 1 year", "1 year to < 2 years","2 years to < 3 years", 
                                        "3 years to < 5 years", "> 5 years"]]).dropna()
ok_sex_wait

In [ ]:
# Plot above on a bar chart

In [ ]:
# Creating payment type vs wait time dataframe

ok_payment_wait = pd.DataFrame(ok_totals1[["Payment type","< 30 days", "30 to < 90 days", "90 days to < 6 months", 
                                        "6 months to < 1 year", "1 year to < 2 years","2 years to < 3 years", 
                                        "3 years to < 5 years", "> 5 years"]]).dropna()
ok_payment_wait

In [ ]:
# Plot above on a bar chart

In [ ]:
# Creating ethnicity vs wait time dataframe

ok_ethnicity_wait = pd.DataFrame(ok_totals1[["Ethnicity","< 30 days", "30 to < 90 days", "90 days to < 6 months", 
                                        "6 months to < 1 year", "1 year to < 2 years","2 years to < 3 years", 
                                        "3 years to < 5 years", "> 5 years"]]).dropna()
ok_ethnicity_wait

In [ ]:
# Plot above on a bar chart

In [ ]:
# Creating age vs wait time dataframe

ok_age_wait = pd.DataFrame(ok_totals1[["Age","< 30 days", "30 to < 90 days", "90 days to < 6 months", 
                                        "6 months to < 1 year", "1 year to < 2 years","2 years to < 3 years", 
                                        "3 years to < 5 years", "> 5 years"]]).dropna()
ok_age_wait

In [ ]:
# Plot above on a bar chart

In [ ]:
#Statistical Analysis for wait times of each bloodtype: mean, median, variance, standard deviation, standard error of the mean (SEM)
ok_blood_type_wait = kidney_data.groupby("Blood type")
